In [7]:
import pandas as pd
import numpy as np
import os

# Подготовка

In [5]:
file=pd.read_excel('БД/Справочник первичных данных.xlsx',dtype=str)
file=file[file['Статус в работе']=='1']['ID'].values[0]
zhzu=pd.read_csv('БД/Первичные данные/'+file+'.csv',dtype=str)

int_cols=['Сумма заявленного убытка','РЗУ на дату заявления',
          'РЗУ текущеее','Сумма оплаты']
for col in int_cols:
    zhzu[col]=zhzu[col].astype(float)

zhzu['статус']=np.where(zhzu['Дата оплаты'].notna(),
                        np.where(zhzu['РЗУ текущеее']>0,'Частично выпл','выпл'),
                        np.where(zhzu['РЗУ текущеее']>0,'открыто',
                                 np.where(zhzu['Дата отказа'].notna(),'отказ','закрыто')))

zhzu['PrKey']=zhzu['№ убытка']+zhzu['Номер договора']+zhzu['Сумма заявленного убытка'].astype(str)

index=['год заявления','месяц заявления','Линия бизнеса_full','статус']

pivot_count_month=zhzu.pivot_table('PrKey', index=index, aggfunc='count')
pivot_sum_rzu_tek_month=zhzu.pivot_table('РЗУ текущеее', index=index, aggfunc='sum')
pivot_sum_rzu_month=zhzu.pivot_table('РЗУ на дату заявления', index=index,aggfunc='sum')
pivot_sum_oplata_month=zhzu.pivot_table('Сумма оплаты', index=index, aggfunc='sum')

zhzu=zhzu.drop(columns=['PrKey'])

pivot_count_month=pivot_count_month.reset_index().rename(columns={'PrKey':'Значение'})
pivot_count_month['Метка']='Кол-во'

pivot_sum_rzu_tek_month=pivot_sum_rzu_tek_month.reset_index().rename(columns={'РЗУ текущеее':'Значение'})
pivot_sum_rzu_tek_month['Метка']='РЗУ текущеее'

pivot_sum_rzu_month=pivot_sum_rzu_month.reset_index().rename(columns={'РЗУ на дату заявления':'Значение'})
pivot_sum_rzu_month['Метка']='РЗУ на дату заявления'

pivot_sum_oplata_month=pivot_sum_oplata_month.reset_index().rename(columns={'Сумма оплаты':'Значение'})
pivot_sum_oplata_month['Метка']='Сумма оплаты'

pivot=pd.concat([pivot_count_month,pivot_sum_rzu_tek_month,
                 pivot_sum_rzu_month,pivot_sum_oplata_month]).reset_index(drop=True)

pivot.to_csv('БД/Отчет 1/1-'+file[2:]+'.csv',index=False)

In [4]:
file

'0-221111'

In [12]:
#Обновление справочника
refbook=pd.read_excel('БД/Справочник отчетов.xlsx',dtype=str)

others_refbook=refbook[refbook['Тип отчета']!='1']
this_refbook=refbook[refbook['Тип отчета']=='1']

this_refbook['Статус в работе']='0'

new_data=pd.DataFrame([['1-'+file[2:],'1','Аналитика убытков',file,
                        file[2:][-2:]+'.'+file[2:][-4:-2]+'.'+file[2:][:2],'1','1']],
                      columns=refbook.columns)

this_refbook=pd.concat([this_refbook,new_data]).reset_index(drop=True)

if this_refbook.shape[0]<=3: pass
else:
    old_refbook=this_refbook[:-3]
    new_refbook=this_refbook[-3:]
    old_refbook['Наличие на сервере']='0'
    del_file=old_refbook[-1:].values[0][0]+'.csv'
    os.remove('БД/Отчет 1/'+del_file)
    this_refbook=pd.concat([old_refbook,new_refbook]).reset_index(drop=True)

refbook=pd.concat([this_refbook,others_refbook]).reset_index(drop=True)
refbook.to_excel('БД/Справочник отчетов.xlsx',index=False)

C:\Users\lucav\AppData\Local\Temp/ipykernel_9648/773879235.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  this_refbook['Статус в работе']='0'
C:\Users\lucav\AppData\Local\Temp/ipykernel_9648/773879235.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  old_refbook['Наличие на сервере']='0'


# Проверка

In [13]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [15]:
file=pd.read_excel('БД/Справочник отчетов.xlsx',dtype=str)
file=file[(file['Тип отчета']=='1')&\
          (file['Статус в работе']=='1')]['ID'].values[0]
pivot=pd.read_csv('БД/Отчет 1/'+file+'.csv',dtype=str)

In [17]:
pivot['Значение']=pd.to_numeric(pivot['Значение'], downcast='float')
pivot['месяц заявления']=pivot['месяц заявления'].apply(lambda x: '0'+x if len(x)==1 else x)
pivot=pivot.sort_values(['год заявления','месяц заявления'])
pivot['Период']=pivot['год заявления']+'/'+pivot['месяц заявления']

df_count=pivot[pivot['Метка']=='Кол-во']
df_now=pivot[pivot['Метка']=='РЗУ текущеее']
df_declared=pivot[pivot['Метка']=='РЗУ на дату заявления']
df_paid=pivot[pivot['Метка']=='Сумма оплаты']

def b_line_part_maker(df_full, b_line='01 - КАСКО'):
    #Получение строк по линии бизнеса
    df_temp=df_full[df_full['Линия бизнеса_full']==b_line]
    df_temp=df_temp.pivot_table(values='Значение',columns='статус',
                                index='Период',aggfunc='sum')
    cols=['выпл','открыто','Частично выпл','закрыто','отказ']
    nocols=[col for col in cols if col not in df_temp.columns]
    for col in nocols:
        df_temp[col]=''
    df_temp=df_temp[['выпл','открыто','Частично выпл','закрыто','отказ']]
    df_temp=df_temp.drop(columns=nocols)
    return df_temp.fillna(0)

figs_data=[[df_count,'Количество на месяц'],
           [df_now,'РЗУ на текущий месяц'],
           [df_declared,'РЗУ на дату заявления'],
           [df_paid,'Оплачено на текущий момент']]

color_discrete_map={'закрыто': 'red','выпл': 'green','открыто': 'gold',
                    'отказ': 'cornflowerblue','Частично выпл': 'cyan'}
category_orders={'статус':['выпл','открыто','Частично выпл','закрыто','отказ']}

figs=[]
for fig in figs_data:
    data,title=fig
    fig_data=b_line_part_maker(data)
    fig=px.bar(fig_data,y=fig_data.columns,title=title,
               color_discrete_map=color_discrete_map,
               category_orders=category_orders,text_auto='.2s')
    fig=go.FigureWidget(fig)
    figs.append(fig)

fig_1,fig_2,fig_3,fig_4=figs

In [18]:
import ipywidgets as ipy
from ipywidgets import Output, VBox, widgets

In [19]:
def response(change):
    fig_dfs=[[fig_1,df_count],[fig_2,df_now],[fig_3,df_declared],[fig_4,df_paid]]
    for fig_df in fig_dfs:
        fig=fig_df[0]; df_temp=fig_df[1]
        nowDF = b_line_part_maker(df_temp, LOB_filter.value)
        nowTrace = px.bar(nowDF,y=nowDF.columns, color_discrete_map=color_discrete_map,
                          category_orders=category_orders,
                          text_auto='.2s')    
        with fig.batch_update():
            fig.data = []
            for tr in nowTrace.data: fig.add_bar(x=tr.x,y=tr.y,name=tr.name,
                                                 marker=tr.marker,texttemplate=tr.texttemplate)
                
#marker сохраняет цвета колонок

LOB_options=pivot['Линия бизнеса_full'].unique()
LOB_options.sort()

LOB_filter = widgets.Dropdown(description='Filter:',value=LOB_options[0],
                             options=LOB_options)

LOB_filter.observe(response, names="value")
    
widgets.VBox([widgets.HBox([LOB_filter]),
              fig_1,fig_2,fig_3,fig_4])